<a href="https://colab.research.google.com/github/ntienvu/KnownOptimum_BO/blob/master/run_all_benchmark_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!sudo apt-get install python3.8.13


!pip install 'scipy==1.9.1' #--force-reinstall
!pip install 'numpy==1.23.3' #--force-reinstall
!pip install 'scikit-learn==1.1.2' #--force-reinstall

!pip install git+https://github.com/ntienvu/KnownOptimum_BO

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scipy-1.9.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (43.4 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.23.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.2
    Uninstalling numpy-1.24.2:
      Successfully uninstalled numpy-1.24.2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ntienvu/KnownOptimum_BO to /tmp/pip-req-build-i8pt0ng3
  Running command git clone --filter=blob:none --quiet https://github.com/ntienvu/KnownOptimum_BO /tmp/pip-req-build-i8pt0ng3
  Resolved https://github.com/ntienvu/KnownOptimum_BO to commit dc60d005c940330821888571ff81efbf2ae411a4
  Preparing metadata (setup.py) ... done


# Demo on benchmark functions

In [2]:
import matplotlib.pyplot as plt
from bayes_opt import BayesOpt_KnownOptimumValue,BayesOpt
import numpy as np
from bayes_opt import vis_ERM,functions,utilities
import warnings
from tqdm.notebook import tqdm


warnings.filterwarnings("ignore")


In [3]:
# Check numpy and scipy version

import scipy
import sklearn

print(np.__version__)
print(scipy.__version__)
print(sklearn.__version__)

# !pip install 'scipy>=1.9.1,<1.9.2'
# !pip install 'numpy>=1.23.1,<1.23.4'
# !pip install scikit-learn==1.1.2


import sys
print('Python version: ', sys.version)


1.24.2
1.9.1
1.1.2
Python version:  3.8.10 (default, Nov 14 2022, 12:59:47) 
[GCC 9.4.0]


In [4]:

import scipy
import sklearn

print("sklearn.__version__",sklearn.__version__)
print("np.__version__",np.__version__)
print("scipy.__version__",scipy.__version__)

sklearn.__version__ 1.1.2
np.__version__ 1.24.2
scipy.__version__ 1.9.1


# Select a function to run the experiment

In [5]:
#myfunction=functions.branin()
#myfunction=functions.hartman_3d()
#myfunction=functions.ackley(input_dim=5)
myfunction=functions.alpine1(input_dim=5)
#myfunction=functions.gSobol(a=np.array([1,1,1,1,1]))
#myfunction=functions.hartman_6d()

print("func:",myfunction.name)


func: alpine1


# Select a list of acquisition functions to compare


In [6]:
acq_type_list=[]

temp={}
temp['name']='erm' # expected regret minimization
temp['IsTGP']=1 # recommended to use tgp for ERM
acq_type_list.append(temp)

temp={}
temp['name']='cbm' # confidence bound minimization
temp['IsTGP']=1 # recommended to use tgp for CBM
acq_type_list.append(temp)

temp={}
temp['name']='mes_fstar' # MES+f*
temp['IsTGP']=0 # we can try 'tgp'
acq_type_list.append(temp)

temp={}
temp['name']='ucb' # GP-UCB
temp['IsTGP']=0 # we can try 'tgp' by setting it =1
#acq_type_list.append(temp)

temp={}
temp['name']='ei' # EI
temp['IsTGP']=0 # we can try 'tgp' by setting it =1
acq_type_list.append(temp)

temp={}
temp['name']='random' # random
temp['IsTGP']=0 # we can try 'tgp' by setting it =1
acq_type_list.append(temp)

# Run the experiments across different acquisition functions

---



In [ ]:
fig=plt.figure()

color_list=['r','b','k','m','c','g']
marker_list=['s','x','o','v','^','>']

for idx, acq_type in enumerate(acq_type_list):

    print("==================acquisition type",acq_type)
    
    IsTGP=acq_type['IsTGP']
    acq_name=acq_type['name']
    
    nRepeat=10
    
    ybest=[0]*nRepeat
    ElapseTime=[0]*nRepeat
    OptTime=[0]*nRepeat
    bo=[0]*nRepeat
   
    for ii in tqdm(range(nRepeat)): # run over multiple repetitions
        
        if 'fstar' in acq_name or acq_name == 'erm' or acq_name == 'cbm':
            bo[ii]=BayesOpt_KnownOptimumValue(myfunction.func,myfunction.bounds,\
                                myfunction.fstar,acq_name,IsTGP,verbose=0)
        else:
            bo[ii]=BayesOpt(myfunction.func,myfunction.bounds,acq_name,verbose=0)
  
        if acq_name =='random': # we generate random points without running BO
            ybest[ii],ElapseTime[ii]=utilities.run_experiment(bo[ii],
                 n_init=13*myfunction.input_dim,NN=0,runid=ii)   
        else:
            ybest[ii],ElapseTime[ii]=utilities.run_experiment(bo[ii],
                                        n_init=3*myfunction.input_dim,\
                                        NN=10*myfunction.input_dim,runid=ii)   
                                       
        OptTime[ii]=bo[ii].time_opt # record the computational time
        print(ybest[ii])
        print("#repeat={} Best Found Value={:.3f}".format(ii,myfunction.ismax*np.max(ybest[ii])))                                              

    # record the result and save to pickle files================================
    # Score={}
    # Score["ybest"]=ybest
    # Score["ElapseTime"]=ElapseTime
    # Score["OptTime"]=OptTime
    # utilities.print_result_sequential(bo,myfunction,Score,acq_type) 
    
    # plot the result ==========================================================
    y_best_sofar=[0]*len(bo)
    for uu,mybo in enumerate(bo):
        y_best_sofar[uu]=[ (myfunction.fstar - np.max(mybo.Y_ori[:ii+1]) ) for ii in range(len(mybo.Y_ori))]
        y_best_sofar[uu]=y_best_sofar[uu][3*myfunction.input_dim-1:] # remove the random phase for plotting purpose
        
    y_best_sofar=np.asarray(y_best_sofar)
    xaxis=range(y_best_sofar.shape[1])
    plt.errorbar(xaxis,np.mean(y_best_sofar,axis=0), np.std(y_best_sofar,axis=0)/np.sqrt(nRepeat),
                 label=acq_type['name'],color=color_list[idx],marker=marker_list[idx])
    
plt.ylabel("Simple Regret",fontsize=14)
plt.xlabel("Iterations",fontsize=14)
plt.legend(prop={'size': 14})
strTitle="{:s} D={:d}".format(myfunction.name,myfunction.input_dim)
plt.title(strTitle,fontsize=18)


==================acquisition type {'name': 'erm', 'IsTGP': 1}


  0%|          | 0/10 [00:00<?, ?it/s]

[ -8.61413684 -14.39890465 -16.93622744 -22.92938243 -11.70222065
 -13.01072522 -13.49332877 -12.68894879  -8.78322918 -13.00487336
 -15.15409797 -13.9441531  -17.24393558 -20.47452169  -6.75988881
  -8.12591663 -14.4691833   -8.34799886 -13.17659305  -9.54674599
 -14.77858797 -15.09772184 -12.84096656 -14.42981191  -6.34477184
 -19.82893063 -12.8205736  -11.14367591  -9.36844237  -9.69781816
 -17.22602777 -14.3888357  -11.29692387 -14.31678723  -9.75417079
 -14.70815062 -11.50502707  -6.92499077  -8.26683218 -10.38006889
 -12.72297762 -10.35643007 -20.00785518 -13.08294999 -14.23909396
  -6.57380853  -5.43109558 -14.59170237  -9.83190462  -6.22860732
  -5.75854061  -4.79031633  -7.95241654 -17.93494219  -6.42029875
 -19.01913607  -9.57681676 -14.60718916  -9.72954983 -12.17335199
 -10.53414051 -13.73876498  -5.70799417 -19.54113844 -19.82862204]
#repeat=0 Best Found Value=4.790
